The goal of this homework is to familiarize users with monitoring for ML batch services, using PostgreSQL database to store metrics and Grafana to visualize them.

In [13]:
import requests
import datetime
import pandas as pd

from evidently import ColumnMapping
from evidently.report import Report
from evidently.metrics import ColumnDriftMetric, DatasetDriftMetric, DatasetMissingValuesMetric , DataDriftTable , ColumnQuantileMetric

from joblib import load, dump
from tqdm import tqdm

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

### Q1. Prepare the dataset

In [2]:
files = [('green_tripdata_2024-03.parquet', './data')]

print("Download files:")
for file, path in files:
    url=f"https://d37ci6vzurychx.cloudfront.net/trip-data/{file}"
    resp=requests.get(url, stream=True)
    save_path=f"{path}/{file}"
    with open(save_path, "wb") as handle:
        for data in tqdm(resp.iter_content(),
                        desc=f"{file}",
                        postfix=f"save to {save_path}",
                        total=int(resp.headers["Content-Length"])):
            handle.write(data)

Download files:


green_tripdata_2024-03.parquet: 100%|██████████| 1372372/1372372 [00:12<00:00, 112999.16it/s, save to ./data/green_tripdata_2024-03.parquet]


In [3]:
march_2024_data = pd.read_parquet('data/green_tripdata_2024-03.parquet')

In [4]:
march_2024_data.shape

(57457, 20)

### Q2. Metric

Let's expand the number of data quality metrics we’d like to monitor! Please add one metric of your choice and a quantile value for the "fare_amount" column (quantile=0.5).

In [5]:
march_2024_data.describe().T

,count,mean,min,25%,50%,75%,max,std
VendorID,57457.0,1.877334,1.0,2.0,2.0,2.0,2.0,0.328056
lpep_pickup_datetime,57457,2024-03-16 04:02:52.405399,2008-12-31 23:02:24,2024-03-08 13:53:56,2024-03-15 22:49:01,2024-03-23 20:11:25,2024-04-01 00:01:45,NaN
lpep_dropoff_datetime,57457,2024-03-16 04:21:00.076039,2008-12-31 23:02:30,2024-03-08 14:13:49,2024-03-15 23:09:52,2024-03-23 20:34:48,2024-04-01 16:11:00,NaN
RatecodeID,55360.0,1.179986,1.0,1.0,1.0,1.0,99.0,1.356719
PULocationID,57457.0,95.524688,1.0,74.0,75.0,97.0,265.0,57.285088
DOLocationID,57457.0,138.629149,1.0,74.0,138.0,220.0,265.0,76.295346
passenger_count,55360.0,1.309538,0.0,1.0,1.0,1.0,9.0,0.967749
trip_distance,57457.0,13.522828,0.0,1.1,1.79,3.1,125112.2,770.416255
fare_amount,57457.0,17.313474,-295.08,9.3,13.5,19.8,841.6,14.958249
extra,57457.0,0.904472,-2.5,0.0,0.0,1.0,10.0,1.382446


In [6]:
def preprocess_data(df): 
    df = df.copy()
    df["duration_min"] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration_min = df.duration_min.apply(lambda td : float(td.total_seconds())/60)


    # filter out outliers
    df = df[(df.duration_min >= 0) & (df.duration_min <= 60)]
    df = df[(df.passenger_count > 0) & (df.passenger_count <= 8)]
    return df

In [7]:
# data labeling
target = "duration_min"
num_features = ["passenger_count", "trip_distance", "fare_amount", "total_amount"]
cat_features = ["PULocationID", "DOLocationID"]

In [8]:
#load model we saved in the training notebook
model = load('models/lin_reg.bin')

# preprocess the March 2024 data
current = preprocess_data(march_2024_data)

# Make predictions and attach them to the dataframe
current["prediction"] = model.predict(current[num_features + cat_features])

In [10]:
# Refernce data for evidently to compare against
reference_data = pd.read_parquet("data/reference.parquet")
reference_data["prediction"] = model.predict(reference_data[num_features + cat_features])

In [ ]:
report = Report(metrics=[
    DataDriftTable(),
    ColumnDriftMetric(column_name="prediction"),    
    ColumnQuantileMetric(column_name="fare_amount", quantile=0.50),  # median fare    
    ColumnQuantileMetric(column_name="passenger_count", quantile=0.50)  # median passengers
])

For the reference dataset i trained it on Jan 2025 data . It has a column called cbd_congestion_fee which is not present in the current data . So we have to drop the extra columns 

In [17]:
# Align reference and current data columns
common_cols = list(set(reference_data.columns) & set(current.columns))
# Keep only common columns in both datasets
reference_data = reference_data[common_cols]
current = current[common_cols]

In [19]:
column_mapping = ColumnMapping(
    prediction="prediction",
    numerical_features=num_features,
    categorical_features=cat_features,
    )

In [20]:
report.run(reference_data=reference_data, current_data=current, column_mapping=column_mapping)

In [33]:
report = report.as_dict()

I chose to find the median number of passenger using the column quantile metric 

In [37]:
for metric_result in report["metrics"]:
    if (
        metric_result["metric"] == "ColumnQuantileMetric"
        and metric_result["result"]["column_name"] == "passenger_count"
        and metric_result["result"]["quantile"] == 0.50
    ):
        current_value = metric_result["result"]["current"]["value"]
        print("Current median:", current_value)      

Current median: 1.0


### Q3. Monitoring

Let’s start monitoring. Run expanded monitoring for a new batch of data (March 2024).

What is the maximum value of metric quantile = 0.5 on the "fare_amount" column during March 2024 (calculated daily)?

In [38]:
# Evidently dashboard
from evidently.metric_preset import DataDriftPreset, DataQualityPreset
from evidently.ui.workspace import Workspace
from evidently.ui.dashboards import DashboardPanelCounter, DashboardPanelPlot, CounterAgg, PanelValue, PlotType, ReportFilter
from evidently.renderers.html_widgets import WidgetSize

In [39]:
ws = Workspace("workspace")

In [40]:
project = ws.create_project("NYC Taxi Data Quality Project Homework")
project.description = "Homework - 5: NYC Taxi Data Quality Project"
project.save()

Project(id=UUID('019783cd-cf59-7e81-b7be-278d91285de8'), name='NYC Taxi Data Quality Project Homework', description='Homework - 5: NYC Taxi Data Quality Project', dashboard=DashboardConfig(name='NYC Taxi Data Quality Project Homework', panels=[], tabs=[], tab_id_to_panel_ids={}), team_id=None, org_id=None, date_from=None, date_to=None, created_at=datetime.datetime(2025, 6, 18, 11, 9, 52, 729847), version='1')

In [48]:
from collections import namedtuple

DailyReport = namedtuple("DailyReport", ["date", "report", "median"])

daily_reports = []

for i in range(1, 31):
    day_start = pd.Timestamp(f'2024-03-{i:02}')
    day_end = pd.Timestamp(f'2024-03-{i+1:02}' if i < 31 else '2024-04-01')
    
    day_data = current[current['lpep_pickup_datetime'].between(day_start, day_end)]
    if day_data.empty:
        continue

    report = Report(metrics=[
        ColumnQuantileMetric(column_name="fare_amount", quantile=0.50)
    ])
    report.run(reference_data=None, current_data=day_data)
    result = report.as_dict()
    
    daily_reports.append(
        DailyReport(
            date=day_start.date(),
            report=report,
            median=result["metrics"][0]["result"]["current"]["value"]
        )
    )
    
# Final result
print(f"Maximum daily median fare: {max(daily_quantiles):.1f}")


Maximum daily median fare: 14.2


### Q4. Dashboard

Finally, let’s add panels with new added metrics to the dashboard. After we customize the dashboard let's save a dashboard config, so that we can access it later. Hint: click on “Save dashboard” to access JSON configuration of the dashboard. This configuration should be saved locally.

Where to place a dashboard config file?

project_folder/dashboards is where the dashboards config file should be placed